### Python scripts to retrieve tables from MYSQL

In [1]:
import pymysql
import sys
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
DB = "iss_project"
PORT = int(3306)
USER = "iss"
PASSWORD = "6Jg3bwm56xtJ2mrfNQwvsaY$"
HOST = "idm5peipdsus5o.crcvo0yw3sz7.ap-southeast-1.rds.amazonaws.com"

In [51]:
connection = pymysql.connect(user=USER, password=PASSWORD, host=HOST, port=PORT, database=DB)
cursor     = connection.cursor()

In [44]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'iss_project'""")
for table in cursor.fetchall():
    print(table)

('inventory',)
('orders',)
('orders_products',)
('orders_products_v2',)
('product_variants_color',)
('product_variants_size',)
('product_variants_version',)
('products',)
('top20forecasts_ARIMA',)


#closing database connection.
if(connection):
    cursor.close()
    connection.close()
    print("MySQL connection is closed")

#### Retrieve products table

In [54]:
cursor.execute("SELECT column_name from INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'products' AND TABLE_SCHEMA = 'iss_project' ORDER BY ordinal_position")
col_name = cursor.fetchall()
column_names = [x[0] for x in col_name]; column_names

['id', 'title', 'sku', 'price_usd', 'category', 'tags', 'created_at']

In [55]:
', '.join(column_names)

'id, title, sku, price_usd, category, tags, created_at'

In [57]:
products = "SELECT id, title, sku, price_usd, category, tags, created_at FROM products"
cursor.execute(products)
products_records = cursor.fetchmany(3)

pd.DataFrame(products_records, columns=column_names)

,id,title,sku,price_usd,category,tags,created_at
0,33184,"MONO Classic Acoustic/Dreadnought Guitar Case,...",M80-AD-BLK,229.99,Acoustic Guitar Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14
1,33186,"MONO Vertigo Electric Guitar Case, Black",M80-VEG-BLK,249.99,Electric Guitar Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14
2,33187,"MONO Classic Electric Guitar Case, Black",M80-EG-BLK,229.99,Electric Guitar Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14


#### Retrieve orders table

In [63]:
cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders' AND TABLE_SCHEMA = 'iss_project' ORDER BY ordinal_position ")
col_name = cursor.fetchall()
pd.DataFrame(products_records, columns=column_names)

['id',
 'created_at',
 'closed_at',
 'processed_at',
 'total_price',
 'financial_status',
 'fulfillment_status',
 'line_items',
 'fulfillments',
 'refunds',
 'discount_applications',
 'customer']

In [70]:
orders_query = "select * from orders"
cursor.execute(orders_query)
orders_records = cursor.fetchmany(5)
pd.DataFrame(orders_records, columns=column_names)

,id,created_at,closed_at,processed_at,total_price,financial_status,fulfillment_status,line_items,fulfillments,refunds,discount_applications,customer
0,20570,2018-01-01 17:24:47,2018-01-02 18:02:30,2018-01-01 17:24:47,449.97,paid,fulfilled,"[{""title"":""Vertigo™ Semi-Hollow"",""quantity"":1,...","[{""created_at"":""2018-01-02T18:02:30-08:00"",""li...",[],[],"{""id"":149426569237,""created_at"":""2018-01-01T17..."
1,20556,2018-01-02 10:56:19,2018-01-02 18:02:32,2018-01-02 10:56:19,249.99,paid,fulfilled,"[{""title"":""Vertigo™ Semi-Hollow"",""quantity"":1,...","[{""created_at"":""2018-01-02T18:02:32-08:00"",""li...",[],[],"{""id"":150321627157,""created_at"":""2018-01-02T10..."
2,20554,2018-01-02 11:45:26,2018-01-02 18:02:33,2018-01-02 11:45:26,NaN,paid,fulfilled,"[{""title"":""The FlyBy"",""quantity"":1,""sku"":""EFX-...","[{""created_at"":""2018-01-02T18:02:33-08:00"",""li...",[],"[{""type"":""manual"",""value"":""229.99"",""value_type...","{""id"":150383820821,""created_at"":""2018-01-02T11..."


#### Retrieve inventory table

In [71]:
cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'inventory' AND TABLE_SCHEMA = 'iss_project' ")
col_name = cursor.fetchall()
column_names = [x[0] for x in col_name]; column_names

['date', 'ending_quantity', 'product_sku']

In [72]:
inventory_query = "select * from inventory"
cursor.execute(inventory_query)
inventory_records = cursor.fetchmany(5)
pd.DataFrame(inventory_records, columns=column_names)

,date,ending_quantity,product_sku
0,CVL-LPD-WHT,3,2018-10-31
1,M80-ST-BLK,3,2018-10-31
2,PFX-PB-LP-SLV,3,2018-10-31
3,M80-UT-BLK,3,2018-10-31
4,M80-VEB-BLK,3,2018-10-31


#### Retrieve orders_products table

In [73]:
cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders_products' AND TABLE_SCHEMA = 'iss_project' ORDER BY ordinal_position ")
col_name = cursor.fetchall()
column_names = [x[0] for x in col_name]

query = "select * from orders_products"
cursor.execute(query)
records = cursor.fetchmany(5)
pd.DataFrame(records, columns=column_names)

,order_id,order_created_at,order_total_price,order_status,order_discounts,product_title,product_sku,product_quantity,product_discount,product_category,product_tags,product_created_at,customer_id,customer_country,customer_created_at
0,20570,2018-01-01 17:24:47,449.97,fulfilled,yes,"MONO Vertigo Semi-Hollow Guitar Case, Black",M80-VHB-BLK,1,0.00,Electric Guitar Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14,149426569237,United States,2018-01-02 01:23:26
1,20570,2018-01-01 17:24:47,449.97,fulfilled,yes,"MONO Classic Tick Accessory Case 2.0, Black",M80-TICK-V2-BLK,1,0.00,Pedals & Pedalboards Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14,149426569237,United States,2018-01-02 01:23:26
2,20570,2018-01-01 17:24:47,449.97,fulfilled,yes,"MONO Pedalboard Lite, Silver",PFX-PB-LT-SLV,1,0.00,Pedalboards,Pedals & Pedalboards,2017-10-26 02:57:14,149426569237,United States,2018-01-02 01:23:26
3,20556,2018-01-02 10:56:19,249.99,fulfilled,yes,"MONO Vertigo Semi-Hollow Guitar Case, Black",M80-VHB-BLK,1,0.00,Electric Guitar Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14,150321627157,United States,2018-01-02 18:55:03
4,20554,2018-01-02 11:45:26,NaN,fulfilled,no,"MONO Classic FlyBy Backpack, Black",EFX-FLY-BLK,1,229.99,DJ Equipment Cases & Gig Bags,Cases & Gig Bags,2017-10-26 02:57:14,150383820821,United States,2018-01-02 19:45:05


#### Retieve Top20forecasts ARIMA Model

In [75]:
cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'top20forecasts_ARIMA' AND TABLE_SCHEMA = 'iss_project' ORDER BY ordinal_position ")
col_name = cursor.fetchall()
column_names = [x[0] for x in col_name]

query = "select * from top20forecasts_ARIMA"
cursor.execute(query)
records = cursor.fetchall()
pd.DataFrame(records, columns=column_names)

,month,test,predict,item
0,m+1,2,9,EFX-FLY-BLK
1,m+2,2,0,EFX-FLY-BLK
2,m+3,2,2,EFX-FLY-BLK
3,m+1,3,3,M80-2B-BLK
4,m+2,1,0,M80-2B-BLK
5,m+3,1,3,M80-2B-BLK
6,m+1,5,3,M80-2G-BLK
7,m+2,5,0,M80-2G-BLK
8,m+3,4,2,M80-2G-BLK
9,m+1,7,0,M80-AC-BLK
